In [694]:
import pandas as pd
import numpy as np
from numpy import nan


In [695]:
df = pd.read_csv("jobs18.csv")

In [696]:
df.head()

,Unnamed: 0,title,company,salary,location,company_rating,description
0,0,Junior Data Scientist (NYC or Remote),ViacomCBS,NaN,"Remote in New York, NY+1 location",3.9,ViacomCBS (NASDAQ:\nVIAC; VIACA) is a leading ...
1,1,"Associate Data Scientist, Content Analytics",Spotify,NaN,Remote in Oregon+12 locations,4.3,"Data, Research & Insights\nData Science\nSpoti..."
2,2,"Data Scientist, TikTok Creation and Consumption",TikTok,NaN,"San Francisco, CA+34 locations",3.5,TikTok is the leading destination for short-fo...
3,3,"Data Scientist, Game Analytics & Strategy",the NBA,NaN,"New York, NY 10022 (Midtown area)+1 location",4.2,"At the NBA, we’re passionate about growing and..."
4,4,Data Scientist and Analytics Specialist,Delta,NaN,"Atlanta, GA+4 locations",4.2,"United States, Georgia, Atlanta\nProduct Manag..."


In [697]:
df.shape

(450, 7)

We will start to work from salary column. Goal here is to have 
1) mean value for the salary if they have a salary estimation
2) or leave it as NaN

First Let's figure out
- How many NaN value we have
- How many value that we can use
- structure of non NaN values
- extract only numbers

In [698]:
print('we have', df['salary'].isna().sum(), 'Nan value for the salary column')
print('that indicates that we have only 56 values to use')

we have 394 Nan value for the salary column
that indicates that we have only 56 values to use


In [699]:
salary = df['salary']
salary_notnull = salary[salary.notnull()]
print(salary_notnull)


11           $110,000 - $140,000 a year
12            $72,272 - $176,300 a year
17            $92,700 - $160,000 a year
21                      $110,500 a year
23            $83,690 - $160,000 a year
26            $70,400 - $117,400 a year
29            $76,000 - $148,000 a year
33            $83,690 - $160,000 a year
35           $120,000 - $223,000 a year
36            $72,272 - $176,300 a year
39     Estimated $47.7K – $60.3K a year
41            $90,000 - $150,000 a year
46            $90,000 - $150,000 a year
55            $86,389 - $116,788 a year
57            $70,400 - $117,400 a year
58                       $95,000 a year
69            $65,072 - $117,129 a year
71           $125,000 - $175,000 a year
72            $90,000 - $150,000 a year
75           $118,248 - $127,208 a year
93                       $98,000 a year
96             $4,372 - $10,792 a month
104           $90,000 - $161,000 a year
112           $90,000 - $161,000 a year
117           $96,400 - $144,600 a year


Now let's extract numbers only from the salary columns

In [700]:
def str_contains_n(str):
    num_str = "0123456789"
    word_list = []
    words = str.split(" ")
    for word in words:
        word_has_n = False
        for char in word:
            if char in num_str:
                word_has_n = True
        if word_has_n:
            word_list.append(word)
    result = '-'.join(word_list)
    return result

In [701]:
df['k_unit'] = df['salary'].apply(lambda x: 1 if isinstance(x, str) and 'k' in x.lower() else 0)
df['yearly'] = df['salary'].apply(lambda x: 1 if isinstance(x, str) and 'a year' in x.lower() else 0)
df['monthly'] = df['salary'].apply(lambda x: 1 if isinstance(x, str) and 'a month' in x.lower() else 0)
df['hourly'] = df['salary'].apply(lambda x: 1 if isinstance(x, str) and 'an hour' in x.lower() else 0)

salary = salary.apply(lambda x: str_contains_n(x) if isinstance(x, str) else None)
salary = salary.apply(lambda x: x.lower().replace("$","").replace("k","").replace(",","") if x != None else None)
salary = salary.apply(lambda x: x if isinstance(x,str) else 0)

df['salary_edit'] = salary

df['min_salary'] = salary.apply(lambda x: float(x.split('-')[0]) if isinstance(x,str) else 0)
df['max_salary'] = salary.apply(lambda x: float(x.split('-')[1]) if isinstance(x,str) and len(x.split('-')) >1 else(float(x.split('-')[0]) if isinstance(x,str) else 0))
df['avg_salary'] = (df['min_salary'] + df['max_salary'])/2

# take care of k_unit
df.loc[df['k_unit'] == 1,['min_salary']] = df[df['k_unit'] == 1].min_salary * 1000
df.loc[df['k_unit'] == 1,['max_salary']] = df[df['k_unit'] == 1].max_salary * 1000
df.loc[df['k_unit'] == 1,['avg_salary']] = df[df['k_unit'] == 1].avg_salary * 1000
df.loc[df['k_unit'] == 1]

# take care of monthly, for the monthly we have to multiply 12
df.loc[df['monthly'] == 1,['min_salary']] = df[df['monthly'] == 1].min_salary * 12
df.loc[df['monthly'] == 1,['max_salary']] = df[df['monthly'] == 1].max_salary * 12
df.loc[df['monthly'] == 1,['avg_salary']] = df[df['monthly'] == 1].avg_salary * 12
df.loc[df['monthly'] == 1]

df.loc[df['hourly'] == 1,['min_salary']] = df[df['hourly'] == 1].min_salary * 40 * 52
df.loc[df['hourly'] == 1,['max_salary']] = df[df['hourly'] == 1].max_salary * 40 * 52
df.loc[df['hourly'] == 1,['avg_salary']] = df[df['hourly'] == 1].avg_salary * 40 * 52
df.loc[df['hourly'] == 1]

# df.at[df['k_unit'] == 1, 'Product_Name'] = df[df['k_unit'] == 1].min_salary * 1000
# Now that we have get the yearly average salary correctly we can drop all unneccessary columns

df.drop(['k_unit','yearly','monthly','hourly','salary_edit','min_salary','max_salary','salary'], axis=1, inplace = True)
# pd.set_option("display.max_rows", None, "display.max_columns", None)




Now we need to clean up the location columns
there are few information we can extract
1. remote or not
2. state name

1) we can compare whether remote job gives more salary or the onsite job
2) which state gives are more salary
3) even with the remote job, does the state gives out high salary still gives are high salary

the above questions we can answer later on

In [702]:

df['remote'] = df['location'].apply(lambda x: 1 if isinstance(x,str) and 'remote' in x.lower() else 0)
df.groupby('remote').count()
# we can see that 158/450 jobs are remote

,Unnamed: 0,title,company,location,company_rating,description,avg_salary
remote,,,,,,,
0,292,292,292,292,251,174,292
1,158,158,158,158,119,121,158


In [703]:
# Let's extract state name
# from https://gist.github.com/JeffPaine/3083347 I copied the state acronymn list
states_ac = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

states_dict = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

states = []
for state in states_dict:
    states.append(states_dict[state])

states_all = states_ac + states



In [704]:
import re
newData = "GeeksforGeeks, is_an-awesome ! app + too"
  
# To split "+" use backslash
print(re.split(', |_|-|!|\+', newData))

def check_contains_ele(location, state_list):
    location = location.replace('in',"")
    location_list = re.split(' |\+',location)
    print(location_list)
    for location in location_list:
         for state in state_list:
             if location.lower() == state.lower():
                 return state
    return 0
    
df['state'] = df['location'].apply(lambda x: check_contains_ele(x, states_all) if isinstance(x,str)  else 0)
df[df['state'] == 0]

['GeeksforGeeks', 'is', 'an', 'awesome ', ' app ', ' too']
['Remote', '', 'New', 'York,', 'NY', '1', 'location']
['Remote', '', 'Oregon', '12', 'locations']
['San', 'Francisco,', 'CA', '34', 'locations']
['New', 'York,', 'NY', '10022', '(Midtown', 'area)', '1', 'location']
['Atlanta,', 'GA', '4', 'locations']
['Hybrid', 'remote', '', 'Seattle,', 'WA', '2', 'locations']
['Boston,', 'MA']
['San', 'Jose,', 'CA', '95101', '(North', 'Valley', 'area)']
['Plano,', 'TX', '75024', '1', 'location']
['Jericho,', 'NY', '11753', '1', 'location']
['New', 'York,', 'NY']
['', '3', 'locationsRemote']
['Washgton,', 'DC', '20505', '(Foggy', 'Bottom', 'area)']
['Remote', '', 'Cummg,', 'GA', '3', 'locations']
['Stone', 'Mounta,', 'GA', '4', 'locations']
['Stone', 'Mounta,', 'GA', '4', 'locations']
['Atlanta,', 'GA', '30349', '5', 'locations']
['', '126', 'locationsRemote']
['Atlanta,', 'GA', '32', 'locations']
['Remote', '', 'New', 'York,', 'NY', '3', 'locations']
['Remote', '', 'California']
['Remote', ''

,Unnamed: 0,title,company,location,company_rating,description,avg_salary,remote,state
11,11,Green Energy Data Scientist,Piper Companies,+3 locationsRemote,4.5,Piper Companies is seeking a Green Energy Data...,125000.0,1,0
17,17,Data Scientist I,Amazon.com Services LLC,+126 locationsRemote,3.5,"M.S. or equivalent in Statistics, Mathematics,...",126350.0,1,0
52,52,Data Scientist,RenoFi,Remote,NaN,As a Data Scientist you'll work with the team ...,0.0,1,0
66,66,Junior Data Scientist,GMR Marketing,Remote,4.2,Â :\nWHY YOU'LL LOVE THIS JOB //\nOur digital ...,0.0,1,0
82,82,Data Scientist,A+E Networks,Remote,3.8,"Contractors\nCurrently, all staff and non-staf...",0.0,1,0
86,86,Sr. Data Scientist - Remote Work,Caterpillar,Remote in Illinois+1 location,3.8,Career Area:\nInformation Analytics\nJob Descr...,0.0,1,0
93,93,Data Scientist,Lime,Remote,3.3,Lime is the world's largest shared electric ve...,98000.0,1,0
94,94,Principal Data Scientist (Remote Friendly),Mission Lane,Virginia+1 location•Remote,3.7,Mission Lane LLC is a fintech on a mission (no...,0.0,1,0
97,97,Applied Data Scientist (Remote),Leaf Logistics,"New York, NY•Remote",NaN,We are Leaf\nLeaf is on a mission to upend the...,0.0,1,0
104,104,Data Scientist/ Data Science Analyst I,Liberty Mutual Insurance,Remote,3.6,"At Liberty Mutual, our purpose is to help peop...",125500.0,1,0


In [705]:
# python
# aws
# gcp
# excel
# spark

df['python'] = df['description'].apply(lambda x: 1 if isinstance(x,str) and 'python' in x.lower() else 0)
df['aws'] = df['description'].apply(lambda x: 1 if isinstance(x,str) and 'aws' in x.lower() else 0)
df['gcp'] = df['description'].apply(lambda x: 1 if isinstance(x,str) and 'gcp' in x.lower() else 0)
df['excel'] = df['description'].apply(lambda x: 1 if isinstance(x,str) and 'excel' in x.lower() else 0)
df['spark'] = df['description'].apply(lambda x: 1 if isinstance(x,str) and 'spark' in x.lower() else 0)

df.columns
df.drop('Unnamed: 0',axis = 1, inplace = False)
df.to_csv('job_data_cleaned.csv')